# Convolutional network

Here, we examine the performance of CNN's on our dataset. The goal is to examine performance in situation where graph information is not refined, that is we keep all FC links.

In [9]:
import numpy as np 
import pandas as pd 
import pickle

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Load data

In [3]:
DATA_FOLDER = '../data'
PICKLE_FOLDER = '../pickles'

In [4]:
with open(f'{PICKLE_FOLDER}/fc-pearson.pickle', 'rb') as f:
    fc_pearson = pickle.load(f)

In [6]:
with open(f'{PICKLE_FOLDER}/fc-spearman.pickle', 'rb') as f:
    fc_spearman = pickle.load(f)

In [15]:
df_metadata = pd.read_csv(f'{DATA_FOLDER}/patients-cleaned.csv', index_col=0)

In [17]:
df_metadata.head(2)

,age,sex,target
0,24.750,1,0
1,27.667,1,0


## Split data

In [21]:
with open(f'{PICKLE_FOLDER}/test-indices.pickle', 'rb') as f:
    test_indices = pickle.load(f)

### Select the dataset to work with.

In [178]:
fc = fc_pearson.copy()

In [192]:
total_samples, total_brain_regions, _ = fc.shape

In [193]:
X_test = fc[test_indices]
y_test = df_metadata.iloc[test_indices]["target"]

In [194]:
train_indices = ~np.isin(np.arange(total_samples), test_indices)
X_train = fc[train_indices]
y_train = df_metadata.iloc[train_indices]["target"]

We will be using LOOCV to get validation performance of our models.

## Prepare data

In [195]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cuda


In [196]:
X_train = torch.from_numpy(X_train).unsqueeze(1).to(device, dtype=torch.float32)
X_test = torch.from_numpy(X_test).unsqueeze(1).to(device, dtype=torch.float32)

y_train = torch.from_numpy(y_train.values).unsqueeze(1).to(device, dtype=torch.float32)
y_test = torch.from_numpy(y_test.values).unsqueeze(1).to(device, dtype=torch.float32)

In [197]:
trainloader = torch.utils.data.DataLoader([*zip(X_train, y_train)], batch_size=4, shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader([*zip(X_test, y_test)], batch_size=4, shuffle=False, num_workers=0)

## Define model

In [237]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)

        self.fc1 = nn.Linear(in_features=1 * 87 * 87, out_features=1)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.view(-1, 1 * 87 * 87)

        x = torch.sigmoid(self.fc1(x))
        return x

In [238]:
net = ConvNet()
net.to(device)

ConvNet(
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(3, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=7569, out_features=1, bias=True)
)

In [239]:
optimizr = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.BCELoss()

## Model training

In [240]:
for epoch in range(200):
    running_loss = 0.

    for i, (images, labels) in enumerate(trainloader, 0):

        optimizr.zero_grad()
        outputs = net.forward(images)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizr.step()

        running_loss += loss.item()
        if i % 10 == 9:
            print(f'[{epoch+1}, {i+1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')


[1,    10] loss: 0.682
[1,    20] loss: 0.697
[1,    30] loss: 0.695
[2,    10] loss: 0.692
[2,    20] loss: 0.692
[2,    30] loss: 0.693
[3,    10] loss: 0.691
[3,    20] loss: 0.689
[3,    30] loss: 0.685
[4,    10] loss: 0.691
[4,    20] loss: 0.683
[4,    30] loss: 0.676
[5,    10] loss: 0.693
[5,    20] loss: 0.684
[5,    30] loss: 0.679
[6,    10] loss: 0.680
[6,    20] loss: 0.653
[6,    30] loss: 0.697
[7,    10] loss: 0.680
[7,    20] loss: 0.674
[7,    30] loss: 0.677
[8,    10] loss: 0.664
[8,    20] loss: 0.650
[8,    30] loss: 0.686
[9,    10] loss: 0.646
[9,    20] loss: 0.657
[9,    30] loss: 0.647
[10,    10] loss: 0.628
[10,    20] loss: 0.666
[10,    30] loss: 0.611
[11,    10] loss: 0.643
[11,    20] loss: 0.616
[11,    30] loss: 0.579
[12,    10] loss: 0.598
[12,    20] loss: 0.611
[12,    30] loss: 0.609
[13,    10] loss: 0.540
[13,    20] loss: 0.593
[13,    30] loss: 0.533
[14,    10] loss: 0.576
[14,    20] loss: 0.467
[14,    30] loss: 0.386
[15,    10] loss: 0

In [241]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        outputs = net(images)

        predicted = torch.where(outputs.view(-1) > 0.5, 1, 0)

        total += labels.size(0)
        correct += (predicted == labels.view(-1)).sum().item()

print(f'Accuracy: {correct / total * 100:.2f} %')

Accuracy: 68.00 %
